# Conformal Prediction Cheat Sheet

This is my summary of what I've learned about how conformal prediction works.

### The Goal

My main goal is to create a **set-valued function**, $\mathcal{C}(X)$, that takes a new example and gives me a set of possible labels. This isn't just any set; it comes with a powerful statistical guarantee. For a given error rate $\alpha$ (e.g., $\alpha=0.1$ for 90% confidence), the set is guaranteed to contain the true label with at least $1-\alpha$ probability.

### The Core Steps

The process works in three main steps, using a **calibration set** of data, $\{(X_i, Y_i)\}_{i=1}^n$, that the model did not see during training.

**1. The Score ($s_i$): Measuring Disagreement**

* First, I need a **score function**, $s(x,y)$, that measures the level of disagreement between my model's prediction for an input $x$ and a label $y$.
* A **high score is bad**. It means the model was "badly wrong". A **low score is good**, representing higher model confidence.
* For the initial classification example, the score for a calibration point is $s_i = 1 - \hat{f}(X_i)_{Y_i}$. Here, $\hat{f}(X_i)_{Y_i}$ is the softmax probability the model assigned to the **true class**.
* I realized this simple score only uses the probability of the true class and seems to "throw away" info from other classes. This is a valid observation.
* However, there are more advanced scores (like for APS) that *do* use the other class probabilities. The choice of score is an important design step.

**2. The Threshold ($\hat{q}$): A Calibrated Ceiling for Error**

* After computing the scores $s_1, ..., s_n$ for every example in my calibration set, I find a threshold, $\hat{q}$.
* This threshold is calculated as the $\frac{\lceil(n+1)(1-\alpha)\rceil}{n}$ quantile of the scores. This is a specific formula that provides the statistical guarantee, not just a simple percentile.
* I now understand $\hat{q}$ as a **calibrated ceiling for model error**. It's the value that separates the bottom ~90% of "more acceptable" disagreement scores from the top ~10% of scores where the model was "most wrong" on the calibration data.

**3. The Prediction Set ($\mathcal{C}(X_{test})$): Testing Every Hypothesis**

* This was the most confusing part for me. For a new test point $X_{test}$, I **don't know the true class**, so I can't directly calculate "the" score.
* The solution is to test **every possible class hypothetically**.
* For each potential label $y$, I calculate a score as if it were the true label, $s(X_{test}, y)$. I then compare this score to my threshold $\hat{q}$.
* My final prediction set, $\mathcal{C}(X_{test})$, is the collection of **all classes whose hypothetical score was less than or equal to $\hat{q}$**.
* The mathematical definition makes this clear: $\mathcal{C}(X_{test}) = \{y : s(X_{test}, y) \le \hat{q}\}$.

### Why It Works: The Guarantee

* The threshold $\hat{q}$ is not arbitrary; it's **calibrated** on my model's actual performance.
* The guarantee comes from **exchangeability**. The score of a new, unseen true label is treated as another random draw from the same distribution as the calibration scores.
* Since ~90% of the true scores in my calibration set were below $\hat{q}$, the new true score also has a ~90% chance of being below $\hat{q}$.
* By building my set to include *every* class that satisfies this condition ($s(X_{test},y) \le \hat{q}$), I create a "safety net" that is guaranteed to catch the true label with at least $1-\alpha$ probability.
* Crucially, this guarantee applies to the **entire set**, not to each individual label within the set.